In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [12]:
import os
import sys
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import argparse
import torch
import torch.nn.init as init
from transformers import (
    AutoTokenizer, LlamaConfig, LlamaForCausalLM, TrainingArguments, Trainer, PreTrainedTokenizerFast, DataCollatorForLanguageModeling, TrainerCallback
)
from datasets import Dataset, DatasetDict
import argparse
import pandas as pd
from huggingface_hub import hf_hub_download

repo = "ai4bharat/sangraha"
sb_folder = "verified/tel"
local_dir = "ai4bharat"

C:\Users\patel\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
def get_size(df):
    return sys.getsizeof(df) / 1024 / 1024

In [14]:
def get_dataset(repo, sb_folder, local_dir, range_):
    telugu_char_pattern = re.compile(r'[^\u0C00-\u0C7F\u0000-\u007F\s<>\n0-9.,!?]')
    filenames = [f"data-{i}.parquet" for i in range(*range_)]
    df_dataset = pd.DataFrame()
    for filename in filenames:
        file_path = hf_hub_download(
            repo_id=repo,
            repo_type="dataset",
            subfolder=sb_folder,
            filename=filename,
            local_dir=local_dir
        )
        df = pd.read_parquet(file_path)
        df = df.rename(columns={"text": "Input"})
        df.drop(columns=['type'], inplace=True)
        df.drop(columns=['doc_id'], inplace=True)
        df["Input"] = "<bos> " + df["Input"] + " <eos>"
        df["Input"] = df["Input"].str.replace("\n", "<newline>")
        df["Input"] = df["Input"].apply(lambda x: telugu_char_pattern.sub("", x))
        df_dataset = pd.concat([df_dataset, df], ignore_index=True)
    
    return df_dataset

In [16]:
df = get_dataset(repo, sb_folder, local_dir, (13, 14))
df

,Input
0,<bos> వెండితెరపై లవర్ బాయ్స్ చాలామంది ఉన్నారు....
1,<bos> ఎల్ఐసీ పాలసీని మధ్యలోనే ఆపేశారా?<newline...
2,<bos> సరికాని ఆహారం మరియు ఉత్పత్తుల నాణ్యమైన న...
3,<bos> మహేష్ సినిమాలో విజయశాంతి పాత్ర ఇదే!<newl...
4,<bos> ఢిల్లీ : భారతీయ రైల్వే మరో ఘనతను సాధించి...
...,...
174757,"<bos> ఎ. వెంక టేశ్వరరావు (విజయవాడ), ఎస్. సత్యన..."
174758,<bos> భారత ప్రధాని హోదాలో అగ్రరాజ్యం అమెరికా ప...
174759,<bos> నవతరం దర్శకుల్లో తనదైన అభిరుచిని చాటుకుం...
174760,<bos> తన భార్య అనుష్క డెలివరీ నేపథ్యంలో విరాట్...


In [17]:
print(f"Original Data Size: {get_size(df)} MB")
df = df[:100000]
print(f"New Data Size: {get_size(df)} MB")

Original Data Size: 710.9850740432739 MB
New Data Size: 406.81468296051025 MB


In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("/kaggle/input/sentencepiecebpe/SentencePieceBPETokenizer_500")

In [8]:
from itertools import chain
def tokenize_function(df, context_length=256):
    
    input_ids = tokenizer(df["Input"].tolist())["input_ids"]
    flattened_ids = list(chain.from_iterable(input_ids))
    
    token_chunks = [
        flattened_ids[i:i + context_length]
        for i in range(0, len(flattened_ids), context_length)
    ]
    
    last_chunk = token_chunks[-1]
    last_chunk_length = len(last_chunk)
    padding_length = context_length - last_chunk_length
    last_chunk += [tokenizer.pad_token_id] * padding_length
    token_chunks[-1] = last_chunk

    return pd.DataFrame({"input_ids": token_chunks})

tokenized_dataset = tokenize_function(df, context_length=256)
tokenized_dataset = Dataset.from_pandas(tokenized_dataset)

In [9]:
print(tokenized_dataset)

Dataset({
    features: ['input_ids'],
    num_rows: 167421
})


In [10]:
print(tokenized_dataset["input_ids"][4])
print()
print(tokenized_dataset["input_ids"][-1])
print()
print(len(tokenized_dataset["input_ids"][-1]))

[6958, 1877, 722, 12949, 2506, 8536, 1343, 314, 3761, 3411, 2659, 5653, 362, 4539, 1899, 259, 5743, 5279, 18936, 3204, 1302, 18867, 2549, 1229, 405, 16773, 1723, 4, 266, 5161, 15085, 10317, 2669, 224, 722, 13447, 11576, 9355, 17425, 722, 1291, 5002, 5108, 2851, 27346, 7039, 8304, 104, 4, 438, 29041, 21523, 5784, 7213, 17944, 2864, 7007, 1909, 29161, 21313, 33, 4, 438, 17918, 11159, 261, 2303, 231, 163, 14627, 33, 4, 438, 6497, 283, 348, 143, 3655, 303, 33, 4, 438, 234, 1171, 25089, 234, 1171, 1252, 407, 1245, 22192, 882, 1189, 22159, 23960, 201, 33, 4, 5862, 5857, 23133, 17470, 8304, 407, 581, 10377, 2793, 5099, 6955, 4204, 15584, 32033, 24073, 1020, 4553, 5071, 2351, 6599, 21881, 3377, 386, 13576, 7764, 10317, 2669, 224, 722, 692, 14292, 393, 4039, 7493, 28, 11455, 4813, 522, 597, 256, 12887, 18, 2659, 5653, 6875, 16792, 3879, 7184, 5555, 14256, 407, 14707, 23508, 13308, 1021, 6875, 512, 3856, 995, 1284, 537, 4235, 1607, 1172, 8066, 294, 3230, 10831, 16469, 7977, 1872, 20839, 5559, 54

In [11]:
llama_config = LlamaConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=512,                       # d_model = 512
    num_hidden_layers=6,                   # L = 6
    num_attention_heads=8,                 # n_h = 8
    intermediate_size=768,
    max_position_embeddings=128,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id
)

model = LlamaForCausalLM(config=llama_config)

In [14]:
for i,j in model.named_parameters():
  if j.requires_grad and len(j.size()) > 1:
    init.xavier_uniform_(j.data)
    
total_param=0
for i,j in model.named_parameters():
    total_param += j.numel()
print(f"Total Number of Parameters: {total_param/(10**6)}M")

Total Number of Parameters: 46.930432M


In [15]:
import math
import wandb

train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

# Data Collator for Language Modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # For causal language modeling
)

wandb.init(project="huggingface", name="llama_telugu")

# Training Arguments
# training_args = TrainingArguments(
#     output_dir="./llama_telugu_model",
#     overwrite_output_dir=True,
#     num_train_epochs=3,  # Number of full training epochs
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     save_steps=500,
#     save_total_limit=2,
#     evaluation_strategy="steps",
#     eval_steps=100,  # Evaluate every 100 steps
#     logging_dir="./logs",
#     logging_steps=50,
#     learning_rate=5e-5,
#     weight_decay=0.01,
#     warmup_steps=500,
#     fp16=True,  # Use mixed precision if possible
# #     report_to=["none"]  # Disable reporting for simplicity
# )

training_args = TrainingArguments(
    output_dir="./results",                # Set a local path to store results (checkpoints)
    evaluation_strategy="steps",           # Evaluate after every epoch
    logging_dir="./logs",                  # Directory for logs
    logging_steps=50,                      # Log every 50 steps
    save_steps=1000,                       # Save checkpoints every 1000 steps
    eval_steps = 1000,
    save_total_limit=3,                    # Limit the number of saved checkpoints
    per_device_train_batch_size=8,         # Adjust batch size based on your memory
    per_device_eval_batch_size=8,
    num_train_epochs=10,                    # Number of epochs
    report_to="wandb",                     # Log to W&B
    logging_first_step=True,
    load_best_model_at_end=True,           # Load the best model based on evaluation
    save_strategy="steps",                 # Save model after every epoch (or steps)
    push_to_hub=False,                       # Avoid pushing to hub (unless needed)
    fp16 = True
)

class PerplexityCallback(TrainerCallback):
    def __init__(self):
        self.perplexity_by_epoch = []

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics and "eval_loss" in metrics:
            perplexity = math.exp(metrics["eval_loss"])
            self.perplexity_by_epoch.append({"epoch": state.epoch, "perplexity": perplexity})
            wandb.log({"eval_perplexity": perplexity})
            
    def on_log(self, args, state, control, logs=None, **kwargs):
        if "eval_loss" in logs:
            perplexity = math.exp(logs["eval_loss"])
            logs["eval_perplexity"] = perplexity
            wandb.log({"eval_perplexity": perplexity})

class ModelSaveCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        # Save the entire model to W&B at the end of each epoch
        model_save_path = f"{wandb.run.dir}/model_epoch_{state.epoch}"
        model.save_pretrained(model_save_path)
        
        # Upload the saved model to W&B
        wandb.save(f"{model_save_path}/*")  # Save all files in the directory to W&B
            
perplexity_callback = PerplexityCallback()

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks=[perplexity_callback, ModelSaveCallback]
)

wandb.watch(model, log="all", log_freq=100)
trainer.train()
wandb.finish()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/c

Step,Training Loss,Validation Loss,Perplexity
1000,8.244300,8.245357,3809.893671
2000,7.776300,7.710895,2232.539624
3000,7.347900,7.343413,1545.979356
4000,7.069000,7.059713,1164.110886
5000,6.833800,6.827794,923.151720
6000,6.694300,6.639542,764.744374
7000,6.508900,6.477914,650.612569
8000,6.334700,6.335208,564.086955
9000,6.232700,6.212095,498.744921
10000,6.089600,6.110009,450.342641


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

eval/loss,█▆▅▄▄▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▃▃▆█▃▂▄▂▄▃▃▆▅▄▂▄▄▄▄▅▃█▄▅▃▂▅▄▄▄▅▃▂▄▁▃▄▄▆▄
eval/samples_per_second,▃▅▃▅▆▅▇▅█▆▄▅▅▆▅▅▄▅▇▃▆▁▄▃▆▆▄▆▅▅▃▅▅▃▃▄▃▆▅▇
eval/steps_per_second,▄▃▅▆▆▅▃█▆▅▅▅▄▄▇▆▃▃▆▁▆▆▅▅▅▇▄▆▅▆▅▅▃▅▆▄▅▄▃▆
eval_perplexity,█▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch,▁▁▁▁▁▁▁▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇██
train/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇█████
train/grad_norm,▁▂▃▃▃▄▄▄▄▅▅▅▅▅▆▅▆▆▆▆▆▆▆▆▆▇▆▇▇▇▇▇█▇▇█▇███
train/learning_rate,████████▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁▁
train/loss,█▆▅▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁
eval/loss,5.03361


In [16]:
print("\nPerplexity by Epoch Fraction:")
perplexity_matrix = pd.DataFrame(perplexity_callback.perplexity_by_epoch)
print(perplexity_matrix)


Perplexity by Epoch Fraction:
       epoch   perplexity
0   0.106180  3809.893671
1   0.212359  2232.539624
2   0.318539  1545.979356
3   0.424719  1164.110886
4   0.530898   923.151720
..       ...          ...
89  9.556169   153.878794
90  9.662349   153.738638
91  9.768528   153.681981
92  9.874708   153.522530
93  9.980888   153.486444

[94 rows x 2 columns]


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "LLM_model_telugu"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
print("Model and tokenizer loaded successfully!")

Model and tokenizer loaded successfully!


In [ ]:
def generate_text(prompt, max_length=50, temperature=1.0, top_k=50, top_p=0.95, repetition_penalty=1.2):
    """
    Generates text based on a given prompt.

    Args:
        prompt (str): The input prompt to generate text from.
        max_length (int): Maximum length of the generated text.
        temperature (float): Sampling temperature for diversity.
        top_k (int): Top-k sampling.
        top_p (float): Top-p (nucleus) sampling.
        repetition_penalty (float): Penalty for token repetition.

    Returns:
        str: The generated text.
    """
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

    output = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

prompts = [
    "<bos> శ్రీకాకుళం జిల్లాలో గిరిజన ప్రజలు తమ జీవనశైలిలో వినియోగించే సంప్రదాయ పద్ధతులు, వాటి వినియోగాలు, మరియు చారిత్రక విశిష్టతలు",
    "<bos> ఆంధ్రప్రదేశ్‌లోని చారిత్రక ప్రదేశాలు మరియు వాటి చరిత్ర, గుట్టల మీద కట్టబడిన దేవాలయాలు",
    "<bos> గోదావరి నది పరివాహక ప్రాంతాలలో రైతులు సాగుచేసే పంటలు, అవి ఎటువంటి పద్ధతిలో సాగుచేయబడతాయి",
    "<bos> భారతదేశం యొక్క వాతావరణ మార్పులు రైతులకు కలిగించే ప్రభావాలు, ఆ ప్రభావాలను తగ్గించడానికి చేపట్టిన చర్యలు",
    "<bos> తెలుగు సాహిత్యంలో వేదకాలం నుండి ఆధునిక యుగం వరకు జరిగిన మార్పులు, ప్రముఖ రచయితలు",
    "<bos> తెలుగు గ్రామీణ ప్రాంతాల్లో ప్రజల జీవన విధానాలు, ముఖ్యమైన వ్యవసాయ పద్ధతులు, గ్రామీణ సంస్కృతి",
    "<bos> సముద్ర తీర ప్రాంతాల్లో చేపల వేట మరియు అక్కడి ప్రజల జీవనశైలి, అందుకు ఉపయోగపడే ఉపకరణాలు",
    "<bos> తెలంగాణలో ప్రసిద్ధమైన బోనాల పండుగ మరియు దాని చరిత్ర, ఉత్సవాల సమయంలో జరిగే ప్రత్యేక కార్యక్రమాలు",
    "<bos> పల్లె జీవనశైలి గురించి తెలుగు సాహిత్యంలో పొందుపరచిన వివరణలు, వాటి ప్రాముఖ్యత",
    "<bos> కృష్ణా నది తీరప్రాంతం గొప్ప వాణిజ్య కేంద్రంగా ఎలా ఎదిగింది, ఆ ప్రాంతంలో జరుగుతున్న వాణిజ్య కార్యక్రమాలు"
]
for prompt in prompts:
    generated_text = generate_text(prompt, max_length=1280, temperature=0.8)
    print(f"Prompt: {prompt}")
    print(f"Generated Text: {generated_text}")
    print()

Prompt: <bos> శ్రీకాకుళం జిల్లాలో గిరిజన ప్రజలు తమ జీవనశైలిలో వినియోగించే సంప్రదాయ పద్ధతులు, వాటి వినియోగాలు, మరియు చారిత్రక విశిష్టతలు
Generated Text: శ్రీకాకుళం జిల్లాలో గిరిజన ప్రజలు తమ జీవనశైలిలో వినియోగించే సంప్రదాయ పద్ధతులు, వాటి వినియోగాలు, మరియు చారిత్రక విశిష్టతలు తదితర అంశాలపై ఆధారపడి ఉన్నారు. ఒక మహిళ మూడు రోజుల పాటు 'నో' అనే పేరుతో నాలుగు వర్గాల నుంచి  వరకు విస్తరించినప్పటికీ ప్రతి సంవత్సరం దాదాపు ఏడాదిపాటు, ఈ ప్రాంతంలో అన్ని ప్రాంతాల్లోనూ కొత్త తరంగాలు నిర్వహిస్తారు. ఆ రెండు అంశాల కంటే భిన్నంగా ఉన్నందున, ఈ ప్రక్రియలో కొన్ని మార్పులు, చేర్పులు చేసేవారు ప్రత్యేక శ్రద్ధ వహిస్తారు. రాష్ట్ర ప్రభుత్వం ఏర్పాటు చేసిన నూతన సచివాలయాలను కలెక్టర్లను కలిసి సమన్వయం చేసుకుని, వారి ఆరోగ్యంతో కూడిన వైద్య సలహాలు, సూచనలు అందించారు. తద్వారా ఆయా జిల్లాల్లోనూ ప్రభుత్వ అనుమతి లేకుండా ప్రజలకు సేవలు అందిస్తున్నది. జిల్లాల వారీగా పోడు భూముల సమస్య గురించి తెలుసుకున్న అధికారులంతా ఈ విషయంలో ముందుకు రాకూడదని అధికారులు చెప్తున్నారు. అలాగే ఇతర ప్రాంతాలకు కూడా నిధులు విడుదల చేయాలని సంబంధిత అధికారులకు, మండల

In [ ]:
prompt = "<bos> గోదావరి నది పరివాహక ప్రాంతాల్లో రైతులు సాగించే పంటలు, వీటిని సాగించడానికి ఉపయోగించే పద్ధతులు, సాగునీటి సరఫరా, మరియు భూగర్భ జలాల వినియోగం అనేవి రైతుల జీవనోపాధి కోసం ముఖ్యమైన అంశాలు. ఈ ప్రాంతంలో రైతులు పంటలు సాగిస్తున్న విధానం, సవాళ్లను ఎదుర్కొనటానికి ప్రభుత్వం చేపడుతున్న చర్యలు, మరియు రాష్ట్ర ప్రభుత్వం ఇటీవల తీసుకున్న చర్యలు, పధకాలు మరియు ప్రాజెక్టుల వివరాలు ఇవ్వడమే కాదు. సాగునీటి పథకాలు, పంటల లాభదాయకత, తదితర అంశాలను కూడా. ప్రస్తుతం, రైతులు ప్రధానంగా"

generated_text = generate_text(prompt, max_length=1280, temperature=0.8)
print(f"Prompt: {prompt}")
print(f"Generated Text: {generated_text}")
print()

Prompt: <bos> గోదావరి నది పరివాహక ప్రాంతాల్లో రైతులు సాగించే పంటలు, వీటిని సాగించడానికి ఉపయోగించే పద్ధతులు, సాగునీటి సరఫరా, మరియు భూగర్భ జలాల వినియోగం అనేవి రైతుల జీవనోపాధి కోసం ముఖ్యమైన అంశాలు. ఈ ప్రాంతంలో రైతులు పంటలు సాగిస్తున్న విధానం, సవాళ్లను ఎదుర్కొనటానికి ప్రభుత్వం చేపడుతున్న చర్యలు, మరియు రాష్ట్ర ప్రభుత్వం ఇటీవల తీసుకున్న చర్యలు, పధకాలు మరియు ప్రాజెక్టుల వివరాలు ఇవ్వడమే కాదు. సాగునీటి పథకాలు, పంటల లాభదాయకత, తదితర అంశాలను కూడా. ప్రస్తుతం, రైతులు ప్రధానంగా
Generated Text: గోదావరి నది పరివాహక ప్రాంతాల్లో రైతులు సాగించే పంటలు, వీటిని సాగించడానికి ఉపయోగించే పద్ధతులు, సాగునీటి సరఫరా, మరియు భూగర్భ జలాల వినియోగం అనేవి రైతుల జీవనోపాధి కోసం ముఖ్యమైన అంశాలు. ఈ ప్రాంతంలో రైతులు పంటలు సాగిస్తున్న విధానం, సవాళ్లను ఎదుర్కొనటానికి ప్రభుత్వం చేపడుతున్న చర్యలు, మరియు రాష్ట్ర ప్రభుత్వం ఇటీవల తీసుకున్న చర్యలు, పధకాలు మరియు ప్రాజెక్టుల వివరాలు ఇవ్వడమే కాదు. సాగునీటి పథకాలు, పంటల లాభదాయకత, తదితర అంశాలను కూడా. ప్రస్తుతం, రైతులు ప్రధానంగా వ్యవసాయం, విద్య, ఉద్యోగావకాశాలు లేక వ్యాధులు ప్రబలకుండా ఉండేంద

## Translation of the above prompt and generated text in English:-



### *Prompt*:- 
- "Crops grown by farmers in the Godavari river basin, farming practices, irrigation water supply, and groundwater use are important factors for farmers' livelihoods. It not only gives details of the way farmers are cultivating crops in the region, steps taken by the government to meet the challenges, and recent measures, schemes and projects taken by the state government. Irrigation schemes, profitability of crops, etc. At present, farmers are the main ones"

### *Generated Text*:-
- Crops grown by farmers in the Godavari river basin, farming practices, irrigation water supply, and groundwater use are important factors for farmers' livelihoods. It not only gives details of the way farmers are cultivating crops in the region, steps taken by the government to meet the challenges, and recent measures, schemes and projects taken by the state government. Irrigation schemes, profitability of crops, etc. At present, farmers are mainly concerned with agriculture, education, employment opportunities or taking appropriate measures to avoid the spread of diseases. All the regions of the district are cultivating in one place, so that the development of the farmers should be taken care of. Especially for the farmers in thousands of hectares across the joint district. Lakh metric tons of grain procurement centers are being set up. Due to this, the government aims to solve the problems related to farmers immediately. There is a need to provide essential seeds and fertilizers for agriculture in the state. As part of this, each village will be subsidized at a higher percentage to increase the production of agricultural products. During the last year so far around Rs. , offering only Rs. At present, the farmers who have high demand in the market are not getting even the minimum support price. Through the Green India Challenge program announced by Telangana Chief Minister KCR.. percent (-) of the total production has been made available. In accordance with this, farmers insurance premium with percentage subsidy of about two lakh crores is being provided in the district over three districts. With the entire paddy harvesting completed in the month of March, plans have been made to move forward with the aim of providing more quality production levels to the people in the upcoming rainy season. The Chief Minister has already started collection of grain in many districts and has decided to focus on its utilization.